# Pose2Sim workshop
EnvisionBOX Summer School, 2025, Amsterdam. Presented by David Pagnon

<bt><br><br>

`Pose2Sim` provides a workflow for 3D markerless kinematics (human or animal), as an alternative to traditional marker-based MoCap methods. 

**Pose2Sim** is free and open-source, requiring low-cost hardware but with research-grade accuracy and production-grade robustness. It gives maximum control over clearly explained parameters. Any combination of phones, webcams, or GoPros can be used with fully clothed subjects, so it is particularly adapted to the sports field, the doctor's office, or for outdoor 3D animation capture.

***Note:*** For real-time analysis with a single camera, please consider **[Sports2D](https://github.com/davidpagnon/Sports2D)** (note that the motion must lie in the sagittal or frontal plane). 


In [ ]:
from IPython.display import Video, display

display(Video('Pose2Sim_small.mp4', embed=True, width=1280, height=720))

# Table of Contents

1. [Install and import libraries](#install-and-import-libraries)
2. [Install Pose2Sim](#install-pose2sim)
    1. [Install miniconda](#install-miniconda)
    2. [Create a new conda environment](#create-a-new-conda-environment)
    3. [Install OpenSim](#install-opensim)
    4. [Install Pose2Sim Core](#install-pose2sim-core)
    5. [Optional: Install GPU libraries](#optional-install-gpu-libraries)
3. [Try Pose2Sim Demo](#try-pose2sim-demo)
    1. [Copy demo files](#copy-demo-files)
    2. [Run Pose2Sim demo](#run-pose2sim-demo)
4. [Run on the Tragic Talkers dataset](#run-on-the-tragic-talkers-dataset)
    1. [Choose a subset](#choose-a-subset)
    2. [Download videos and calibration files](#download-videos-and-calibration-files)
    3. [Convert calibration files](#convert-calibration-files)
    4. [Copy configuration file](#copy-configuration-file)
    5. [Run pose estimation](#run-pose-estimation)
    6. [Run person association and triangulation](#run-person-association-and-triangulation)
    7. [Filter and run marker augmentation](#filter-and-run-marker-augmentation)
    8. [Run scaling and inverse kinematics](#run-scaling-and-inverse-kinematics)

## Install and import libraries

Install and import the libraries required for the notebook:

In [ ]:
!pip install ipywidgets ipyfilechooser opencv-python numpy
!jupyter nbextension enable --py widgetsnbextension

import os
import json
from pathlib import Path
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from ipyfilechooser import FileChooser

try: # For Google Colab
    from google.colab import output
    output.enable_custom_widget_manager()
except ImportError: # For Jupyter Notebook
    pass

## Install Pose2Sim

***N.B.*** *The following instructions are the same as those in the [Pose2Sim documentation](https://github.com/perfanalytics/pose2sim)*
<br>

0.1. Download this notebook.
0.2. Install [miniconda](https://docs.conda.io/en/latest/miniconda.html) if you don't have it already.
0.3. Install and open VSCode, pycharm, or JupyterLab, or anything . 

1. Install [miniconda](https://docs.conda.io/en/latest/miniconda.html)
2. Create a new conda environment:

In [ ]:
!conda create -n Pose2Sim python=3.10 -y 
!conda activate Pose2Sim

3. Install OpenSim:

In [ ]:
!conda install -c opensim-org opensim -y

4. Install Pose2Sim:

In [ ]:
!pip install pose2sim

5. *Optional:* Install the libraries for GPU support:

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip uninstall onnxruntime
!pip install onnxruntime-gpu

## Try Pose2Sim Demo

Copy the demo files to the desired location:

In [ ]:
from Pose2Sim import Pose2Sim

singleperson_demo_path = Path(Pose2Sim.__file__).parent.resolve() / 'Demo_SinglePerson'

def create_folder_picker():
    fc = FileChooser(
        path=Path.cwd(),  # Start in current directory
        filename='',
        title='Select download folder',
        select_desc='Choose',
        change_desc='Change'
    )
    fc.use_dir_icons = True
    fc.filter_pattern = None  # Show all items
    fc.dir_icon = '📁'
    fc.show_only_dirs = True  # Only show directories
    return fc

fc = create_folder_picker()
display(fc)
selected_folder = Path.cwd() if fc.selected is None else Path(fc.selected)

!cp -r {singleperson_demo_path} {selected_folder}/
os.chdir(singleperson_demo_path)

Run Pose2Sim demo:


In [ ]:
Pose2Sim.calibration()
Pose2Sim.poseEstimation()
Pose2Sim.synchronization()
Pose2Sim.personAssociation()
Pose2Sim.triangulation()
Pose2Sim.filtering()
Pose2Sim.markerAugmentation()
Pose2Sim.kinematics()

# Or equivalently:
#Pose2Sim.runAll() 
# Or:
#Pose2Sim.runAll(do_calibration=True, do_poseEstimation=True, do_synchronization=True, do_personAssociation=True, do_triangulation=True, do_filtering=True, do_markerAugmentation=True, do_kinematics=True)

## Run on the Tragic Talkers dataset


### Choose a subset of the dataset to run on, ie a session name and some cameras (2 minimum, preferably 4 or more)

In [ ]:
def create_folder_picker():
    fc = FileChooser(
        path=Path.cwd(),  # Start in current directory
        filename='',
        title='Select download folder',
        select_desc='Choose',
        change_desc='Change'
    )
    fc.use_dir_icons = True
    fc.filter_pattern = None  # Show all items
    fc.dir_icon = '📁'
    fc.show_only_dirs = True  # Only show directories
    return fc

session_name = widgets.Dropdown(
    options=['conversation1_t3', 'femalemonologue1_t2', 'femalemonologue2_t3', 'interactive1_t2', 'interactive4_t3', 'male_monologue2_t3'],
    value='femalemonologue2_t3',
    description='Session name:',
    disabled=False,
)

camera_numbers = widgets.SelectMultiple(
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'],
    value=['01', '11', '12', '22'], 
    rows=12,
    description='Choose cameras',
    disabled=False
)

fc = create_folder_picker()

display(session_name)
display(camera_numbers)
display(fc)

Download videos and calibration files from the chosen subset:

In [ ]:
base_url = 'https://github.com/sarkadava/envisionBOX_SummerschoolAmsterdam2025'

selected_folder = Path.cwd() if fc.selected is None else Path(fc.selected)
session_folder = selected_folder/session_name.value

def download_videos():
    for camera_number in camera_numbers.value:
        file_name = f"{session_name.value}-cam{camera_number}.mp4"
        file_url = f"{base_url}/raw/main/Sample/{session_name.value}/{file_name}"
        (session_folder/'videos').mkdir(parents=True, exist_ok=True)
        os.system(f"wget -O {session_folder/'videos'/file_name} {file_url}")
        print(f"Downloaded: {session_folder/'videos'/file_name}")

def download_calibration():
    for camera_number in camera_numbers.value:
        file_name = f"camera-0{camera_number}.json"
        file_url = f"{base_url}/raw/main/Sample/camera_calibration_data/{file_name}"
        (session_folder/'calibration').mkdir(parents=True, exist_ok=True)
        os.system(f"wget -O {session_folder/'calibration'/file_name} {file_url}")
        print(f"Downloaded: {session_folder/'calibration'/file_name}")

download_videos()
download_calibration()

### Convert the calibration files to the Pose2Sim format

In [ ]:
from Pose2Sim.calibration import toml_write
from Pose2Sim.common import rotate_cam, world_to_camera_persp

selected_folder = Path.cwd() if fc.selected is None else Path(fc.selected)
session_folder = selected_folder/session_name.value

json_calib_files = sorted((session_folder/'calibration').glob('*.json'))
ret, C, S, D, K, R, T = [], [], [], [], [], [], []
for file_path in json_calib_files:
    with open(file_path, 'r') as f:
        calib_data_cam = json.load(f)['camera']
        C.append(file_path.stem)  # Name
        S.append([int(calib_data_cam['width']), int(calib_data_cam['height'])]) # Size
        D.append([float(d) for d in calib_data_cam['distortion']][:4]) # Distortion
        K.append(np.array([[float(calib_data_cam['fx']), float(calib_data_cam['skew']), float(calib_data_cam['cx'])],
                   [0, float(calib_data_cam['fy']), float(calib_data_cam['cy'])],
                   [0, 0, 1]])) # Intrinsics
        rot_mat_cam = np.array([float(r) for r in calib_data_cam['r']]).reshape(3,3)
        t_cam = np.array([float(t) for t in calib_data_cam['t']])

        # Rotate cameras by Pi/2 around x in world frame
        # camera frame to world frame
        R_w, T_w = world_to_camera_persp(rot_mat_cam, t_cam)
        # x_rotate -Pi/2 and z_rotate Pi
        R_w_90, T_w_90 = rotate_cam(R_w, T_w, ang_x=-np.pi/2, ang_y=0, ang_z=np.pi)
        # world frame to camera frame
        R_c_90, T_c_90 = world_to_camera_persp(R_w_90, T_w_90)
        # store in R and T
        R.append(cv2.Rodrigues(R_c_90)[0].squeeze())
        T.append(t_cam)

toml_calib_file = session_folder/'calibration'/f"Calib_{'_'.join(camera_numbers.value)}.toml"
toml_write(toml_calib_file, C, S, D, K, R, T)


Copy the configuration file to the session folder:

In [ ]:
from Pose2Sim import Pose2Sim
singleperson_demo_path = Path(Pose2Sim.__file__).parent.resolve() / 'Demo_SinglePerson'
!cp {singleperson_demo_path}/Config.toml {session_folder}/

Select model with hands and run pose estimation 

N.B.: We do not need to run Pose2Sim.calibration() since we already retrieved and converted the original calibration files.\
N.B.2: We do not need to run Pose2Sim.synchronization() either, since the cameras are natively synchronized.

In [ ]:
import toml

os.chdir(session_folder)
config_dict = toml.load('Config.toml')
config_dict['project']['project_dir'] = session_folder
config_dict['project']['multi_person'] = True
config_dict['pose']['pose_model'] = 'Whole_body'
config_dict['filtering']['display_figures'] = False

Pose2Sim.poseEstimation(config_dict)

In [ ]:
# ## Does not seem to with with the current codec
# from IPython.display import Video
# for vidfile in (session_folder/'pose').glob('*.mp4'):
#     print(f"Video: {vidfile.name}")
#     display(Video(vidfile, embed=True, width=640, height=535))

In [ ]:
Pose2Sim.personAssociation(config_dict)
Pose2Sim.triangulation(config_dict)

Filter and run marker augmentation:

In [ ]:
Pose2Sim.filtering(config_dict)
Pose2Sim.markerAugmentation(config_dict)

Run scaling and inverse kinematics:

In [ ]:
Pose2Sim.kinematics(config_dict)